In [ ]:
import copy
from constant import REPORTDAYS, HEADER_NAME, COLUMNS_TO_DROP
import pandas as pd
import numpy as np
from plot_helper import df_col_replace

In [ ]:
def find_closest_run_to_median_triple(dflist, pattern):
  df_IQR = pd.DataFrame(columns = range(361)) # 361 rows of data are in output file
  for onerun in dflist:
    df_IQR = df_IQR.append(onerun.filter(regex=pattern, axis=1).sum(axis=1), ignore_index=True)
  # calculate 90% CI and IQR
  df_list_median = df_IQR.astype('float').quantile(0.5).values # type: np array
  # check abs difference b/w each run and the median
  abs_diff_list = []
  # go thru 100 separate run
  for i in range(100):
    # calculate distance to median
    abs_diff_list.append(np.sum(np.absolute(df_list_median - df_IQR.iloc[i].values)))
  return np.asarray(abs_diff_list)

In [ ]:
def find_closest_run_to_median_double(dflist_arg, drug):
  option = 1
  dflist = copy.deepcopy(dflist_arg) 
  for i in range(len(dflist)):
    dflist[i] = df_col_replace(dflist[i], drug, option)
  most_dang_double = '2-4'
  if drug == 'DHA-PPQ': # only count '2-2''s
    most_dang_double = '2-2'
  df_IQR = pd.DataFrame(columns = range(361)) # 361 rows of data are in output file
  for onerun in dflist:
    df_IQR = df_IQR.append(onerun[most_dang_double])
  # calculate 90% CI and IQR
  df_list_median = df_IQR.astype('float').quantile(0.5).values # type: np array
  # check abs difference b/w each run and the median
  abs_diff_list = []
  # go thru 100 separate run
  for i in range(100):
    # calculate distance to median
    abs_diff_list.append(np.sum(np.absolute(df_list_median - df_IQR.iloc[i].values)))
  return np.asarray(abs_diff_list)

In [ ]:
def find_closest_run_to_median_five_dang_types(dflist):
  total_abs_diff = find_closest_run_to_median_triple(dflist, 'TYY..Y2.')
  total_abs_diff = total_abs_diff + find_closest_run_to_median_triple(dflist, 'KNF..Y2.')
  total_abs_diff = total_abs_diff + find_closest_run_to_median_double(dflist, 'DHA-PPQ')
  total_abs_diff = total_abs_diff + find_closest_run_to_median_double(dflist, 'ASAQ')
  total_abs_diff = total_abs_diff + find_closest_run_to_median_double(dflist, 'AL')
    
  min_val = np.min(total_abs_diff)
  min_idx = np.argmin(total_abs_diff) + 1 # naming of run is 1-100
  return (min_idx, min_val)

Either find for each group of 100 runs

In [ ]:
filepath = 'reversed_cycling_drug_order/set5_c/monthly/set5c_%smonthly_data_0.txt'

In [ ]:
dflist = []
for i in range(1,101):
  dflist.append(pd.read_csv(filepath % i, index_col=False, names=HEADER_NAME, sep='\t').drop(columns=COLUMNS_TO_DROP).fillna(0))

In [ ]:
find_closest_run_to_median_five_dang_types(dflist)

Or for entire setting

In [ ]:
def onestep(setnum, strategy):
  dflist = []
  for i in range(1,101):
    dflist.append(pd.read_csv(f'raw_data/delayed_drug_switch/set{setnum}{strategy}/monthly/set{setnum}{strategy}_%smonthly_data_0.txt' % i, index_col=False, \
                                  names=HEADER_NAME, sep='\t').drop(columns=COLUMNS_TO_DROP).fillna(0))
  result = find_closest_run_to_median_five_dang_types(dflist)
  print(result)
  return result[0]

In [ ]:
temp = []
for i in range(1,13):
  temp.append(onestep(i, '2ac'))

In [ ]:
len(temp)

In [ ]:
temp